In [1]:
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

## Dataset MNIST

El dataset **MNIST** consiste en imágenes de números manuscritos entre 0 y 9. Las imágenes tienen dimensiones de 28x28 pixeles y cada pixel está representado por un valor de intensidad en escala de grises. El conjunto de entrenamiento consiste en 60000 dígitos y el conjunto de prueba de 10000.

### Implemente una CNN para obtener un desempeño de al menos 99% en el conjunto de prueba

In [2]:
#Importamos las librerias y la base de datos MNIST
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

In [3]:
# Cargar el dataset MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [4]:
# Preprocesamos los datos
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))
train_images, test_images = train_images / 255.0, test_images / 255.0

In [5]:
# Definimos la red neuronal convolucional
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [6]:
# Compilamos el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [13]:
# Entrenamos el modelo
model.fit(train_images, train_labels, epochs=15, batch_size=64)

Epoch 1/15
938/938 [==============================] - 51s 49ms/step - loss: 0.0055 - accuracy: 0.9983
Epoch 2/15
938/938 [==============================] - 27s 28ms/step - loss: 0.0043 - accuracy: 0.9987
Epoch 3/15
938/938 [==============================] - 21s 22ms/step - loss: 0.0049 - accuracy: 0.9984
Epoch 4/15
938/938 [==============================] - 23s 24ms/step - loss: 0.0054 - accuracy: 0.9984
Epoch 5/15
938/938 [==============================] - 25s 26ms/step - loss: 0.0033 - accuracy: 0.9990
Epoch 6/15
938/938 [==============================] - 29s 31ms/step - loss: 0.0059 - accuracy: 0.9980
Epoch 7/15
938/938 [==============================] - 23s 24ms/step - loss: 0.0029 - accuracy: 0.9990
Epoch 8/15
938/938 [==============================] - 23s 25ms/step - loss: 0.0039 - accuracy: 0.9988
Epoch 9/15
938/938 [==============================] - 24s 25ms/step - loss: 0.0040 - accuracy: 0.9987
Epoch 10/15
938/938 [==============================] - 23s 25ms/step - loss: 0.003

In [14]:
# Evaluamos el modelo en el conjunto de prueba
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f'Accuracy en el conjunto de prueba: {test_acc*100:.2f}%')

313/313 [==============================] - 5s 8ms/step - loss: 0.0541 - accuracy: 0.9905
Accuracy en el conjunto de prueba: 99.05%
